In [77]:
import pandas as pd

In [3]:
original_data = pd.read_parquet("escooter_copy.parquet")

### Data Cleaning

In [20]:
df_org_data = original_data.copy()

In [30]:
# Transformed dtypes of Dataframe   
df_org_data.holiday = df_org_data.holiday.astype(bool)
df_org_data.workingday = df_org_data.workingday.astype(bool)
df_org_data.loc[df_org_data.temp >45, "temp"] = df_org_data.temp.div(100)

In [24]:
# Dataframe as no NaN values
df_org_data.isnull().describe()

,datetime,holiday,workingday,weather,temp,atemp,humidity,windspeed,registered_customer
count,3755415,3755415,3755415,3755415,3755415,3755415,3755415,3755415,3755415
unique,1,1,1,1,1,1,1,1,1
top,False,False,False,False,False,False,False,False,False
freq,3755415,3755415,3755415,3755415,3755415,3755415,3755415,3755415,3755415


In [29]:
# Duplicates ??
df_org_data.drop_duplicates().count(axis=0)

datetime               3610562
holiday                3610562
workingday             3610562
weather                3610562
temp                   3610562
atemp                  3610562
humidity               3610562
windspeed              3610562
registered_customer    3610562
dtype: int64

### Temp Fehler

In [4]:
df_temp = original_data.copy()

In [52]:
# Temperatures that are too high
df_temp.loc[df_temp.temp >45]

,datetime,holiday,workingday,weather,temp,atemp,humidity,windspeed,registered_customer
2138,2020-01-06 10:10:53,0.0,1.0,"clear, few clouds",738.0,8.335,43.0,16.9979,True
2139,2020-01-06 10:13:12,0.0,1.0,"clear, few clouds",738.0,8.335,43.0,16.9979,False
2140,2020-01-06 10:13:43,0.0,1.0,"clear, few clouds",738.0,8.335,43.0,16.9979,True
2141,2020-01-06 10:14:04,0.0,1.0,"clear, few clouds",738.0,8.335,43.0,16.9979,True
2142,2020-01-06 10:16:25,0.0,1.0,"clear, few clouds",738.0,8.335,43.0,16.9979,True
...,...,...,...,...,...,...,...,...,...
3751388,2022-01-03 10:18:44,0.0,1.0,"cloudy, mist",820.0,10.605,69.0,8.9981,True
3751389,2022-01-03 10:18:51,0.0,1.0,"cloudy, mist",820.0,10.605,69.0,8.9981,True
3751390,2022-01-03 10:18:59,0.0,1.0,"cloudy, mist",820.0,10.605,69.0,8.9981,False
3751391,2022-01-03 10:19:11,0.0,1.0,"cloudy, mist",820.0,10.605,69.0,8.9981,True


In [49]:
df_temp.loc[(df_temp.temp >45) & (df_temp.temp.isnull() == True)]

,datetime,holiday,workingday,weather,temp,atemp,humidity,windspeed,registered_customer


In [74]:
temp_mistake = df_temp.loc[df_temp.temp >45].groupby(pd.Grouper(key="datetime", freq="S")).agg({"weather": "count","humidity": "mean","windspeed": "mean","workingday": "sum","holiday": "sum", "temp": "mean", "atemp": "mean", "registered_customer": "count"}).dropna()

In [76]:
# 10:00:00 AM is the time
temp_mistake.sample(50)

,weather,humidity,windspeed,workingday,holiday,temp,atemp,registered_customer
datetime,,,,,,,,
2021-09-06 10:15:36,1,74.0,15.0013,0.0,1.0,3034.0,35.605,1
2020-03-30 10:10:22,1,30.0,12.9980,1.0,0.0,984.0,11.365,1
2021-08-16 10:10:37,1,40.0,0.0000,1.0,0.0,3116.0,34.090,1
2021-12-06 10:10:28,1,77.0,8.9981,1.0,0.0,1804.0,21.970,1
2020-05-11 10:17:22,1,52.0,12.9980,1.0,0.0,2296.0,26.515,1
2021-08-09 10:17:02,1,70.0,15.0013,1.0,0.0,3034.0,34.850,1
2021-11-22 10:13:30,1,66.0,16.9979,1.0,0.0,1476.0,16.665,1
2021-02-22 10:16:21,1,55.0,30.0026,0.0,1.0,902.0,9.850,1
2020-06-01 10:13:00,1,55.0,16.9979,0.0,1.0,3280.0,37.880,1
